### Importing the file and storing data in pandas dataframe

In [1]:
import pandas as pd

# importing parallel programming libraries
from multiprocessing import Lock, Process
import multiprocessing

df = pd.read_csv('GlobalLandTemperaturesByCity.csv/GlobalLandTemperaturesByCity.csv')

FileNotFoundError: [Errno 2] File GlobalLandTemperaturesByCity.csv/GlobalLandTemperaturesByCity.csv does not exist: 'GlobalLandTemperaturesByCity.csv/GlobalLandTemperaturesByCity.csv'

### Creating a series dataframe of the distinct countries and storing them in a list 

In [ ]:
#getting only the countries of the dataset
countries = df['Country'].unique()

# copying the countries in a list
l=[]
for c in countries:
    l.append(c)

### Parallel code to compute and show the average temperature of each country

In [ ]:
def compute_mean(l, listn):
        
    for pays in listn:
        moyenne = (df.loc[df['Country'] == pays]).AverageTemperature.mean()
        l.acquire()
        try:
            print('Average of ', pays, ' is', moyenne)
        finally:
            l.release()
        

if __name__ == "__main__":  # confirms that the code is under main function
    
    lock = Lock() # equvalent of mutex variable in c++, to synchrnoize the display 
    
    thread_max = multiprocessing.cpu_count() # getting the maximum threads that can be used

    n_countries = df.Country.nunique() # getting the number of countries

    count = int((df.Country.nunique())/thread_max)+1 
    
    #spliting the great list l in another great list L containing 8 lists of lists
    i = 0
    L = []
    while (i<n_countries):
        L.append(l[i:(i+count)])
        i = i + count
        
    procs = []
    proc = Process(target=compute_mean)  # instantiating without any argument
    procs.append(proc)
    proc.start()
    
    proc_id = 0
# instantiating process with arguments
    for lst in L:
        proc = Process(target=compute_mean, args=(lock, lst))
        procs.append(proc)
        proc.start()
        print('process', proc_id, 'started')
        proc_id = proc_id+1
        
    proc_id = 0
# complete the processes
    for proc in procs:
        print('process', proc_id, 'ended')
        proc_id = proc_id+1
        proc.join()
    

### Parallel code to show each country, its maximum AverageTemperature, its city and its date

In [ ]:
from multiprocessing import Lock, Process
import multiprocessing

sub_dataset = df[['AverageTemperature', 'Country', 'City', 'dt']]
max_c = df.groupby(['Country'])['AverageTemperature'].max()

i=0
decpt=0
M=[]
while (i<n_countries):
        M.append(max_c.iloc[i:i+20])
        i = i + 20
        decpt +=1


df2 = pd.DataFrame(columns=['AverageTemperature','Country','City','dt'])

def compute_maxi(l, dframe1, dframe2, liste):
    global df2
    for i in range(len(dframe2)):
        df2 = df2.append(dframe1.loc[(dframe1['AverageTemperature']==dframe2.values[i])&(dframe1['Country']==dframe2.index[i])])
    print(df2)
    #print(df2[['AverageTemperature', 'Country']])
        
if __name__ == "__main__":  # confirms that the code is under main function
    
    lock = Lock() # equvalent of mutex variable in c++, to synchrnoize the display 
    
    thread_max = multiprocessing.cpu_count() # getting the maximum threads that can be used

    n_countries = df.Country.nunique() # getting the number of countries

    count = int((df.Country.nunique())/thread_max)+1 
           
    #spliting the great list l in another great list L containing 8 lists of lists
    i = 0
    L = []
    while (i<n_countries):
        L.append(l[i:(i+count)])
        i = i + count
    
    procs = []
    proc = Process(target=compute_maxi)  # instantiating without any argument
    procs.append(proc)
    proc.start()
    
    proc_id = 0
# instantiating process with arguments
    for lst in L:
        proc = Process(target=compute_maxi, args=(lock, sub_dataset, M[L.index(lst)], lst))
        procs.append(proc)
        proc.start()
        print('process', proc_id, 'started')
        proc_id = proc_id+1
        
    proc_id = 0
# complete the processes
    for proc in procs:
        print('process', proc_id, 'ended')
        proc_id = proc_id+1
        proc.join()